In [ ]:
# --- Cell 1: Install dependencies ---
!pip install groq

# --- Cell 2: Import libraries ---
import csv
import os
import time
from datetime import datetime
from groq import Groq  # LLM API
from getpass import getpass

# --- Cell 3: Set API key ---

# Option A: Paste key directly (less secure)
# os.environ["GROQ_API_KEY"] = "PASTE_KEY_HERE"

# Option B: Store key in environment variable beforehand
# api_key=os.environ.get("GROQ_API_KEY")

# Option C: input when exectued
api_key = getpass("Paste your Groq API key here: ")

client = Groq(api_key = api_key)

# --- Cell 4: Define experiment conditions ---
conditions = [
    ("TrustGame_Friendly", "You are Player 2 in a trust game. Player 1 sends you 10 tokens, tripled to 30. How many do you return if you feel friendly?"),
    ("TrustGame_Business", "You are Player 2 in a trust game. Player 1 sends you 10 tokens, tripled to 30. How many do you return if you are focused on maximizing your profit?"),
]

repetitions = 3
model_name = "llama3-8b-8192"
output_file = "experiment_results.csv"

# --- Cell 5: Define query function ---
def query_llm(prompt):
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=200
    )
    return response.choices[0].message.content.strip()

# --- Cell 6: Run experiment ---
with open(output_file, mode="w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["timestamp", "condition", "prompt", "response"])

    for label, prompt_template in conditions:
        for i in range(repetitions):
            timestamp = datetime.utcnow().isoformat()
            print(f"Running {label}, repetition {i+1}/{repetitions}...")
            try:
                response = query_llm(prompt_template)
            except Exception as e:
                response = f"ERROR: {str(e)}"
            writer.writerow([timestamp, label, prompt_template, response])
            time.sleep(1)  # avoid rate limit issues

print(f"Experiment complete! Results saved to {output_file}")

# --- Cell 7: Preview results ---
import pandas as pd
df = pd.read_csv(output_file)
df